# Initial Screening Layer

In [ ]:
import pandas as pd
jf=pd.read_csv('/content/Jobs.csv',encoding = "ISO-8859-1")
cf=pd.read_csv('/content/Candidate.csv',encoding = "ISO-8859-1")

In [ ]:
jf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2002 entries, 0 to 2001
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Job ID            0 non-null      float64
 1   Date              0 non-null      float64
 2   Company Name      1989 non-null   object 
 3   Post              1989 non-null   object 
 4   Location          1989 non-null   object 
 5   Description       1989 non-null   object 
 6   Level             1989 non-null   object 
 7   Type              1989 non-null   object 
 8   Function          1989 non-null   object 
 9   Industry          1989 non-null   object 
 10  Responsibilities  1989 non-null   object 
 11  Qualifications    1989 non-null   object 
dtypes: float64(2), object(10)
memory usage: 187.8+ KB


In [ ]:
cf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16170 entries, 0 to 16169
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       16170 non-null  int64  
 1   ID               16170 non-null  int64  
 2   About            16080 non-null  object 
 3   Accomplishments  11870 non-null  object 
 4   Education        15900 non-null  object 
 5   Experience       15800 non-null  object 
 6   Industry         0 non-null      float64
 7   Level            16170 non-null  object 
 8   Location         16170 non-null  object 
 9   Skills           15860 non-null  object 
 10  Type             16170 non-null  object 
 11  Certification    8020 non-null   object 
dtypes: float64(1), int64(2), object(9)
memory usage: 1.5+ MB


In [ ]:
# Creating courpus for generating TFIDF vectors (Stacking of jobs and candidates together)
corpus=[]
J_ids=[]
C_ids=[]
for i in range(len(jf)):
  corpus.append(str(jf.iloc[i,4])+' '+str(jf.iloc[i,6])+' '+str(jf.iloc[i,7]))      # taking collumns of level, type and location
  J_ids.append('J'+' '+str(i))
for i in range(len(cf)):
  corpus.append(str(cf.iloc[i,7])+' '+str(cf.iloc[i,8])+' '+str(cf.iloc[i,10]))      # taking collumns of level, type and location
  C_ids.append('C'+' '+str(i))

In [ ]:
'''job_ids=[i for i in range(len(jf))]
cand_ids=[i for i in range(len(cf))]
ids=[]
for i in range(len(job_ids)):
  ids.append('J'+' '+str(i))
for i in range(len(cand_ids)):
  ids.append('C'+' '+str(i))'''

In [ ]:
# Code for TFIDF vectorization, these vectors will be fed in birch algorithm
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
print(X.shape)

['abha', 'al', 'applicable', 'associate', 'contract', 'dammam', 'director', 'entry', 'executive', 'full', 'internship', 'jeddah', 'jizan', 'jubail', 'khobar', 'level', 'madinah', 'makkah', 'mid', 'nan', 'non', 'not', 'riyadh', 'senior', 'tabuk', 'time']
(18172, 26)


In [ ]:
#Code for birch clustering
from sklearn.cluster import Birch
brc = Birch(n_clusters=None) # n_cluster is a hyperparameter, which needs to be decded on basis of size of corpus
brc.fit(X)                   # set n_clusters=None to return subclusters as it is, change n_clusters to some number to perform final clustering step
i_cluster=brc.predict(X)

In [ ]:
ids=J_ids+C_ids

In [ ]:
# output dataset, contating job/candidate id and their cluster number.
cluster_dataframe=pd.DataFrame(list(zip(ids,i_cluster)),columns=['ID','Cluster number'])
cluster_dataframe

,ID,Cluster number
0,J 0,0
1,J 1,0
2,J 2,0
3,J 3,0
4,J 4,0
...,...,...
18167,C 16165,29
18168,C 16166,29
18169,C 16167,29
18170,C 16168,29


In [ ]:
cluster_dataframe.to_excel('Clustered_dataframe.xlsx')